In [20]:
%matplotlib inline
import utils; reload(utils)
from utils import *

In [2]:
x = random((30,2))
y = np.dot(x, [1.,3.]) + 1

In [3]:
lm = Sequential([Dense(1, input_shape=(2,))])

In [4]:
lm.compile(optimizer=SGD(lr=0.1),loss='mse')

In [5]:
lm.evaluate(x,y)

30/30 [==============================] - 0s


10.055233001708984

In [6]:
lm.fit(x,y,nb_epoch=5,batch_size=1)

Epoch 1/5
30/30 [==============================] - 0s - loss: 1.1027     
Epoch 2/5
30/30 [==============================] - 0s - loss: 0.1064     
Epoch 3/5
30/30 [==============================] - 0s - loss: 0.0431     
Epoch 4/5
30/30 [==============================] - 0s - loss: 0.0199     
Epoch 5/5
30/30 [==============================] - 0s - loss: 0.0091     


In [7]:
lm.evaluate(x,y)

30/30 [==============================] - 0s


0.0059807430952787399

In [8]:
lm.get_weights()

[array([[ 0.9483],
        [ 2.7799]], dtype=float32), array([ 1.1627], dtype=float32)]

In [9]:
path = 'data/dogscats/'
#path = path + 'sample/'
model_path = path + 'models/'
if not os.path.lexists(model_path): os.mkdir(model_path)

In [27]:
batch_size=64

In [11]:
from vgg16 import Vgg16
vgg = Vgg16()

In [12]:
model = vgg.model

In [41]:
model

Get true labels for all images

Get 1000 imagenet category pred for each image

Make linear model mapping (1000,1)->(2,1) for imagenet predictions->cats/dogs

In [13]:
batches = get_batches(path + 'train', batch_size=batch_size)
val_batches = get_batches(path + 'valid',shuffle=False,batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [14]:
def save_array(fname,arr): c=bcolz.carray(arr,rootdir=fname,mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [15]:
val_data = get_data(path+'valid')

Found 2000 images belonging to 2 classes.


In [16]:
tr_data = get_data(path+'train')

Found 23000 images belonging to 2 classes.


In [17]:
save_array(model_path + 'tr_data.bc',tr_data)
save_array(model_path + 'val_data.bc',val_data)

In [19]:
tr_data = load_array(model_path + 'tr_data.bc')
val_data = load_array(model_path + 'val_data.bc')

MemoryError: 

In [24]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [25]:
val_classes = val_batches.classes
tr_classes = batches.classes
val_labels = onehot(val_classes)
tr_labels = onehot(tr_classes)

In [28]:
tr_features = model.predict(tr_data,batch_size=batch_size)
val_features = model.predict(val_data,batch_size=batch_size)

In [29]:
save_array(model_path + 'tr_lastlayer_features.bc',tr_features)
save_array(model_path + 'val_lastlayer_features.bc',val_features)

In [30]:
lm = Sequential([Dense(2, activation='softmax',input_shape=(1000,))])

In [31]:
lm.compile(optimizer=RMSprop(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

In [35]:
import gc
gc.collect()

24847

23000000

In [38]:
model.fit_generator(batches, nb_epoch=1, samples_per_epoch=batches.nb_sample, validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Exception: You must compile your model before using it.

In [ ]:
from IPython.display import FileLink
FileLink('Tstlesson2.ipynb')